In [1]:
import yfinance as yf
import pandas as pd
import psycopg as pg

with pg.connect(dbname='Stocks', user='postgres', password='420691') as conn:
    with conn.cursor() as cur:
        cur.execute('SELECT ticker FROM tickers')
        tickers = cur.fetchall()

assets = [ticker[0] for ticker in tickers]

df_list = []
for asset in assets:
    data = yf.download(asset, start='2020-01-01')
    data = data.droplevel(1, axis=1)
    data['ticker'] = asset
    df_list.append(data)

df = pd.concat(df_list, axis=0)

# Data cleaning
mask = (df['High'] < df['Open']) | (df['High'] < df['Close']) | (df['Low'] > df['Open']) | (df['Low'] > df['Close'])
clean = df[~mask].copy()
temp = df[mask].copy()

temp['High'] = temp[['Open', 'Close', 'High']].max(axis=1)
temp['Low'] = temp[['Open', 'Close', 'Low']].min(axis=1)
clean = pd.concat([clean, temp], axis=0)
clean.reset_index(inplace=True)

# Insert data into database
from io import StringIO

# Create buffer
buffer = StringIO()
clean.to_csv(buffer, index=False, header=False)
buffer.seek(0)


with pg.connect(dbname='Stocks', user='postgres', password='420691') as conn:
    # The COPY operation needs to be within a WITH block
    with conn.cursor().copy("COPY daily (date, adj_close, close, high, low, open, volume, ticker) FROM STDIN WITH CSV") as copy:
        copy.write(buffer.getvalue())
    
    conn.commit()